In [2]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [3]:
# Prepare source dataset 

source_name = 'data/stiffness/Stiffness_size_{size}_stiffs_{stiffs}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

k = 2

Config = {
    'size' : 10,
    'stiffs' : [k * 1e0, k * 1e1, k * 1e2, k * 1e3, k * 1e4, k * 1e5, k * 1e6],
    'stiff_count' : 200, 
    'shots' : 1000, 
    'stepsize' : 2.00, 
    'accepterr': [0.01,0.05,0.01], 
    'iterations_limit' : 1000, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'CIA'
}

template = pd.read_csv('data/template.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()


log_file, id = log._load_logfile()


Logfile data/stiffness/Stiffness_size_10_stiffs_[2.0, 20.0, 200.0, 2000.0, 20000.0, 200000.0, 2000000.0]_ansatz_CIA_stepsize_2.0_shots_1000.csv already exists


In [ ]:


for stiff in Config['stiffs']: 
    for i in tqdm.tqdm(range(Config['stiff_count']), desc = f"Generating matrices with stiffness {stiff}"):
        
        id += 1
        matrix = prep.stiffnessPreparator(size = Config['size'], stiffness = stiff)
        current_datetime = datetime.now()
        data = { 
            'Id (int)' : id, 
            'Valid (int)' : 1, 
            'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            'Matrix (mat)' : matrix,
            'Size (int)' : Config['size'],
            'Type (str)' : 'Stiff',
            'Rank (int)' : -1,
            'Density (flt)' : -1.0,
            'Stiffness (flt)' : stiff,
            'Penalty (flt)' : -1.0,
            'Other (flt)' : -1.0,
            'Test_solver (str)' : '-',
            'Test (flt)' : -1.0,
            'Test_state (vec)' : [],
            'Test_iterations (int)' : -1,
            'Test_time (flt)' : -1.0,
            'Train_solver (str)' : '-',
            'Preprocessing (str)' : '-',
            'Postprocessing (str)' : '-',
            'Train (flt)' : -1.0,
            'Train_state (vec)' : [],
            'Train_iterations (int)' : -1, 
            'Train_iterations_1 (int)' : -1,
            'Train_iterations_2 (int)' : -1,
            'Train_iterations_3 (int)' : -1,
            'Train_shots (int)': -1, 
            'Train_cost (flt)' : -1.0,
            'Train_time (flt)' : -1.0, 
            'Flags (str)' : '',
            'Baren (int)' : -1
        }
        
        log.log(data)

    
display(log.read(1))

Generating matrices with stiffness 2000000.0: 100%|██████████| 150/150 [00:47<00:00,  3.17it/s]

No line with id {id} - there is nothing to read


{'Id (int)': 0,
 'Valid (int)': 0,
 'Updated (str)': 0,
 'Matrix (mat)': 0,
 'Size (int)': 0,
 'Type (str)': 0,
 'Rank (int)': 0,
 'Density (flt)': 0,
 'Stiffness (flt)': 0,
 'Penalty (flt)': 0,
 'Other (flt)': 0,
 'Test_solver (str)': 0,
 'Test (flt)': 0,
 'Test_state (vec)': 0,
 'Test_iterations (int)': 0,
 'Test_time (flt)': 0,
 'Train_solver (str)': 0,
 'Preprocessing (str)': 0,
 'Postprocessing (str)': 0,
 'Train (flt)': 0,
 'Train_state (vec)': 0,
 'Train_iterations (int)': 0,
 'Train_iterations_1 (int)': 0,
 'Train_iterations_2 (int)': 0,
 'Train_iterations_3 (int)': 0,
 'Train_cost (flt)': 0,
 'Train_shots (int)': 0,
 'Train_time (flt)': 0,
 'Flags (str)': 0,
 'Baren (int)': 0}

In [27]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])


In [22]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [42]:
# test(range(1,21), solver = 'bruteforce')

In [23]:

ids = range (351,1401)
cores = 10
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[ 351,  352,  353, ...,  453,  454,  455],
       [ 456,  457,  458, ...,  558,  559,  560],
       [ 561,  562,  563, ...,  663,  664,  665],
       ...,
       [1086, 1087, 1088, ..., 1188, 1189, 1190],
       [1191, 1192, 1193, ..., 1293, 1294, 1295],
       [1296, 1297, 1298, ..., 1398, 1399, 1400]], shape=(10, 105))

{'Id (int)': 771, 'Valid (int)': 1, 'Updated (str)': '2025-08-24 17:44:30', 'Matrix (mat)': array([[-0.05075609,  0.03966234,  0.05132007,  0.04450764,  0.05746069,
         0.07660427,  0.1693327 ,  0.1543694 ,  0.08744905, -0.05847467],
       [ 0.03966234, -0.09324341, -0.11228622, -0.12559345, -0.19059711,
        -0.14048493, -0.28364719, -0.24820015, -0.18195364,  0.16414397],
       [ 0.05132007, -0.11228622, -0.16480859, -0.14457937, -0.21899399,
        -0.10439622, -0.41587498, -0.23741887, -0.19889378,  0.15565286],
       [ 0.04450764, -0.12559345, -0.14457937, -0.11254271, -0.17037946,
        -0.14658034, -0.2869438 , -0.19786129, -0.18296321,  0.18354602],
       [ 0.05746069, -0.19059711, -0.21899399, -0.17037946, -0.26667646,
        -0.18844498, -0.52326478, -0.35737434, -0.26205213,  0.19977053],
       [ 0.07660427, -0.14048493, -0.10439622, -0.14658034, -0.18844498,
        -0.1854326 , -0.43217475, -0.27355282, -0.24951041,  0.18821279],
       [ 0.1693327 , -0.28

In [4]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False)
        print(f'Trained id = {line['Id (int)']} with train {train}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [ ]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 10

solvers = [] 

depth = 1

ids = np.array(range(1, 1401))
ids = ids.reshape(cores, len(ids) // cores)

template = pd.read_csv('data/template.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = Config['ansatz'], gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))



log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()

data = [[log.read (id) for id in subrange] for subrange in ids]
#print(train_dataset)
# data = [[log.read(id) for id in range (id_start, id_start + len(train_dataset) // cores)] for id_start in range (1, len(train_dataset), len(train_dataset) // cores) ]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




Trained id = 1261 with train -2.3121446311762854
Trained id = 701 with train -3.9484078108525487
Trained id = 281 with train -3.362220128051847
Trained id = 141 with train -1.9429142009767166
Trained id = 702 with train -10.053840196593809
Trained id = 981 with train -6.20833203806159
Trained id = 282 with train -1.8025469197816357Trained id = 703 with train -4.750217301845298

Trained id = 1 with train -3.2764170661225496
Trained id = 841 with train -2.093364870338943
Trained id = 982 with train -4.955001382031833
Trained id = 561 with train -7.156111118152472
Trained id = 842 with train -3.027642104706001
Trained id = 1121 with train -1.5356660632401662
Trained id = 1262 with train -3.9469343596666304
Trained id = 421 with train -3.6877171930937935
Trained id = 142 with train -3.284358053451807
Trained id = 1122 with train -6.263449627405632
Trained id = 843 with train -0.9698665051808313
Trained id = 562 with train -1.318894156842054
Trained id = 1263 with train -3.0344464128834363


In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)


NameError: name 'processed_data' is not defined